tensowflow는 현재는 2, 버전으로 업그레이드가 되었지만 1, 버전을 알아야 정확한 동작의 흐름을 이해할 수 있고
코드를 작성할 때 조금 더 정확하고 조밀하게 작성을 할 수가 있다.
(현재 tensorflow를 사용하는 회사들도 1 버전으로 사용하고 있는 경우가 대부분이다. 1을 알면 2가 가능하지만 2만 알고있다면 
tensorflor 자체의 흐름을 파악하기 어렵다.)

In [3]:
import tensorflow.compat.v1 as tf  # 이 코드 전체는 2.버전을 1.버전으로 다운그레이드 해주어 사용한다는 import이다.
tf.disable_v2_behavior()

tf.get_default_graph().get_operations() # 이 코드까지 정확하게 작성을 해주어야 1.버전을 이용할 수 있다.

[]

tensorflow는 다른 언어들과 차별화되게 학습을 시키는데 적합한 언어이므로 직접적으로 출력을 하지않고 
아래 출력결과와 같이 출력이 된다. 
operation이란것은 텐서에서 계산을 수행하는 "노드를 나타낸다."

In [4]:
a = tf.constant(7.0 , name = "data1") # const는 상수형태
b = tf.constant(3.0 , name = "data2") 
c = tf.constant(2.0 , name = "data3")

tf.get_default_graph().get_operations()

[<tf.Operation 'data1' type=Const>,
 <tf.Operation 'data2' type=Const>,
 <tf.Operation 'data3' type=Const>]

In [5]:
print(a) # print를 사용해도 아래와 같이 tensor의 내부설명이 출력된다.
print()
print(a.op) # op를 출력시키면 그대로 내부의 모습이 나온다. (진행흐름이 나타나는 것이다.)

Tensor("data1:0", shape=(), dtype=float32)

name: "data1"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 7.0
    }
  }
}



In [6]:
print(a.op.outputs) # a의 입력값을 확인
print(a is a.op.outputs[0]) # 텐서 a 와 a의 operation의 출력이 같은지 확인
print(a.op.node_def.attr["value"].tensor.float_val[0]) # 내부의 tensorfloat val을 출력시켜준다.
# a의 operation protocol 버퍼에서 출력값을 확인해 볼 수 있음 

[<tf.Tensor 'data1:0' shape=() dtype=float32>]
True
7.0


In [9]:
v = a * b / c # 연산을 할당하는 변수를 생성해준다.


tf.get_default_graph().get_operations()

In [10]:
print("v.op.outputs = " , v.op.outputs)  # 아래에 출력값에서 보다시피 속성과 차원 , 그리고 dtype를 출력한다.
print(v.op.outputs[0] is v)  # 보다시피 v의 출력값과 v.op.outputs의 출력값은 같다는 걸 알수있다.

v.op.outputs =  [<tf.Tensor 'truediv:0' shape=() dtype=float32>]
True


In [11]:
print(v.op) # op 내부를 출력한다.

name: "truediv"
op: "RealDiv"
input: "mul"
input: "data3"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}



텐서플로같은 경우는 흐름이다. 타언어처럼 즉각적인 값이 생성되는것이 아니라 연산시간또한 느리다. 
그리고 텐서플로는 계산순서가 정확하게 되어있기 때문에 뭉치계산이 아니라 하나씩 찢어서 계산한 후 다음계산을 하고 이런식으로 연산을 진행한다.

In [12]:
print(v.op.inputs[0]) # v같은 경우는 mul 과 div가 포함되어있는데 a ,b ,c는 각각 재료들이고 mul 과 div는 기계들이다. 
                      # 연산자가 커피머신이고 연산에 사용되는 재료들이 원두라고 생각하면 된다.
print(v.op.inputs[1])

Tensor("mul:0", shape=(), dtype=float32)
Tensor("data3:0", shape=(), dtype=float32)


Session함수를 사용해야 우리가 원하는 출력값을 얻을 수 있다.(2.xx 버전에서는 사용이 불가하다.)
Session함수를 먼저 생성해주고 sess.run함수를 사용한다. rum()안에는 만약 상단에 Variable 함수가 존재하면 
tf.global_variables_initializer() 함수를 반드시 작성해주고 ()안에 기입해야 한다. 
위의 함수를 작성하지 않고 실행을 할때에는 error가 발생한다.

In [15]:
with tf.Session() as sess: # 파이썬의 file을 열때 사용하는 with와 같은 개념이다.
    sess.run(tf.global_variables_initializer())
    print(sess.run(v)) # 위의 2줄의 코드를 작성하고 나서야 sess.run(v)로 우리가 원하는 연산값을 얻을 수 있다.

10.5


텐서는 내용을 변경할수는 없음 새로운 텐서를 만드는것만 가능 
텐서는 Rank , shape , Type 3가지 구성요소를 가지고 있음 

랭크가 0이면 스칼라 / 1이라면 벡터 / 2라면 행렬 / 3이상이라면 n - Tensor 또는 
n차원 텐서 

tf.constant = 상수 , 상수형은 말 그대로 상수를 저장하는 데이터 형 

tf.variable  조건에 따라 값을 바꿔야 하는 경우 사용하는 자료형 
변수는 반드시 명시적으로 초기화 해야함 

tf.placeholder :먼저 만들고 값을 나중에 지정해주는 자료형 
값을 나중에 정의하는 특이한 자료형 

In [27]:
a = tf.constant(5) # constant는 지속적으로 얘기하지만 상수이다.
b = tf.constant(2)
c = tf.constant(3)
d = tf.multiply(a , b) # mul은 곱셉연산자이다. 커피머신이 된다.
e = tf.add(c,d)        # add는 덧셈이다. 마찬가지로 커피머신이다.
f = tf.subtract(d ,e)  # sub은 뺼셈이다.

# 아래와 같이 print를 해주면 tensor가 출력된다. 자세히 살펴보면 속성과 , 차원 , 타입이 출력되는것을 볼 수 있다.
# 타입에 int32는 입력된 숫자가 정수형이라는것을 나타낸다.
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

Tensor("Const_13:0", shape=(), dtype=int32)
Tensor("Const_14:0", shape=(), dtype=int32)
Tensor("Const_15:0", shape=(), dtype=int32)
Tensor("Mul_7:0", shape=(), dtype=int32)
Tensor("Add_4:0", shape=(), dtype=int32)
Tensor("Sub_4:0", shape=(), dtype=int32)


In [29]:
sess = tf.Session() # Session함수 사용자가 원하는 출력값을 출력하기 위한 필수 생성함수  
outs = sess.run(f) # run 함수 ()안의 연산이나 문자 , 정수를 출력해준다.
sess.close() # file을 사용하는것과 같이 항상 close를 해주어야 한다.
print("outs = " , outs)

outs =  -3


In [14]:
hello = tf.constant("Hello , Tensorflow!")  # 이렇게 문자열 또한 출력이 가능하다.
sess = tf.Session() # 실행방식은 같다.
print(sess.run(hello))
sess.close()

b'Hello , Tensorflow!'


In [22]:
input_data = [1,2,3,4,5] # 먼저 input할 데이터를 생성해준다.
x = tf.placeholder(dtype = tf.float32) # placeholder은 최초 생성을 할 때 값을 할당해주지 않고 후에 할당을 해준다.
# 후미에 float를 설정한 이유는 나중에 입력될 값이 float형태로 변경해서 입력되게 하는것이다.
y = x  * 2  

sess = tf.Session()
result = sess.run(y , feed_dict = {x:input_data}) # feed_dict는 딕셔너리이고 value값을 다음과 같이 할당을 해주어야 한다. 
                                 # 값을 그대로 넣는것이 아니라 x에 값을 할당한다음 입력해주는것을 확인할 수 있다.
sess.close()

print(result)

[ 2.  4.  6.  8. 10.]


In [25]:
p_holder1 = tf.placeholder(dtype = tf.float32) # placeholder 함수를 3개 생성 
p_holder2 = tf.placeholder(dtype = tf.float32)
p_holder3 = tf.placeholder(dtype = tf.float32)

val1 = 5   # 이미 알아차렸겠지만 각각의 placeholder함수에 입력될 값이다. 
val2 = 10 
val3 = 3

ret_val = p_holder1 * p_holder2 + p_holder3 # 새로운 변수를 생성해 내용을 작성해준다. 

sess = tf.Session()
# 딕셔너리 변수를 생성해서 각각 값을 할당해준다. 
feed_dict_data = {p_holder1 : val1 , p_holder2 : val2 , p_holder3 : val3} 
# ret_val과 feed_dict 딕셔너리에 value값을 할당해준다. 
result = sess.run(ret_val , feed_dict = feed_dict_data)
sess.close()
print(result)

53.0


In [32]:
I = tf.placeholder(tf.float32 , [2,4])
i = [[1,2,3,4] , [5,6,7,8]]

W = tf.placeholder(tf.float32 , [4,2])
w = [[1,1] , [2,2] , [3,3] , [4,4]]

# input 데이터와 weight를 행렬곱 수행 
m_cal = tf.matmul(I , W)

sess = tf.Session()

print(sess.run(m_cal , feed_dict = {I:i , W:w}))  # I 의 1행과 W의 1열을 곱해준다.(1,1 2 ,2 3,3 ,4,4)(5,1 6,2 7,3 8,4)

sess.close()

[[30. 30.]
 [70. 70.]]


In [28]:
# 위의 식에서는 placeholder을 사용했고 
# 현재식에서는 Variable을 사용했기 때문에 global을 사용했다.
I = tf.Variable([[1,2,3,4] , [5,6,7,8]] , tf.float32)
W = tf.Variable([[1,1] , [2,2] , [3,3] , [4,4]] , tf.float32)

m_cal = tf.matmul(I , W)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print(sess.run(m_cal))

sess.close()

[[30 30]
 [70 70]]


In [33]:
# 처음에는 Variable을 사용했기 때문에 global을 사용했어야 하는데 사용하지 않아서 오류가 발생했다.
# 현재는 오류없이 원하는 연산값이 출력된다. 
input_data = [1,2,3,4,5]
x = tf.placeholder(dtype = tf.float32)
W = tf.Variable([2] , dtype = tf.float32) 
y = W * x

sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)
result = sess.run(y , feed_dict={x : input_data })
print(result)

[ 2.  4.  6.  8. 10.]


In [34]:
x = tf.Variable(5 , dtype = tf.int32)
print(x)

z = tf.assign(x , 7) # x의 값을 7로 새롭게 정의하고 그 내용을 z변수에 저장 
sess = tf.Session()
sess.run(z) #여기서 중요한 포인트는 Variable을 사용했음에도 global을 사용하지 않고도 오류가 발생하지 않았다.
            # 그 이유는 상단에 assign함수를 사용해줌으로서 x의 값을 할당해주었기 때문에 global을 사용하지 않고도 오류가 발생하지 않는것이다.
y = sess.run(x)
print(y)
sess.close()

<tf.Variable 'Variable_5:0' shape=() dtype=int32_ref>
7


In [38]:
x = tf.Variable(tf.zeros((2,2)) , dtype = tf.float32) # [[0,0] , [0,0]] 배열을 생성
print(x)

y = tf.Variable([[1,2] , [3,4]] , dtype = tf.float32)
x = tf.assign(x , y)  # x의 값을 y의 값으로 업데이트
sess = tf.Session()
z = tf.global_variables_initializer() # assign을 사용하고도 global을 사용한 이유는??
sess.run(z)
k = sess.run(x)
sess.close()

print(k)

<tf.Variable 'Variable_8:0' shape=(2, 2) dtype=float32_ref>
[[1. 2.]
 [3. 4.]]


In [46]:
# placeholder은 변수 선언 후 나중에 값을 할당해주어야 하는 변수 
# placeholder 활용 문제 
mat_img = [1,2,3,4,5]
label = [10,20,30,40,50]
ph_img = tf.placeholder(dtype = tf.float32)
ph_lb = tf.placeholder(dtype = tf.float32)

ret_tensor = ph_img + ph_lb

sess = tf.Session()

result = sess.run(ret_tensor , feed_dict={ph_img : mat_img , ph_lb : label} )
sess.close()
print(result)

[11. 22. 33. 44. 55.]


In [47]:
# 배열의 곱
input_data = [[1,2,3] , [1,2,3] , [2,3,4]]
x = tf.placeholder(dtype = tf.float32 , shape = [None , 3]) # 값을 받을떄 객체가 3개인 배열을 받겠다고 정해놓았고 
# 그 배열을 몇개나 받을지는 설정해놓지 않았다.
w = tf.Variable([[2.] , [2.] , [2.]] , dtype = tf.float32)
y = tf.matmul(x , w) # 곱하기 
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
result = sess.run(y , feed_dict = {x : input_data}) # x의 1행과 w의 1열을 곱셈 곱셈을 할때는 같은 인덱스끼리 곱한다.
sess.close()
print(result)

[[12.]
 [12.]
 [18.]]


아래는 브로드캐스팅 문법이다. 2 * 1 , 1 * 2 차원이 같아야 연산이 가능하다. 이 차원이 맞지않을경우는 자연스레 차원이 낮은쪽을
높은쪽의 차원으로 업로드 시켜주는데 이것이 바로 브로드 캐스팅이다.

In [39]:
input_data = [[1,1,1] , [2,2,2]]
x = tf.placeholder(dtype = tf.float32 , shape = [2 , 3])
w = tf.Variable([[2.] , [2.] , [2.]] , dtype = tf.float32)
b = tf.Variable([4] , dtype = tf.float32) 
y = tf.matmul(x , w) + b 
print(x.get_shape())
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
result = sess.run(y , feed_dict = {x : input_data})
sess.close()
print(result)


(2, 3)
[[10.]
 [16.]]


In [49]:
x = tf.constant([[1.0 , 2.0 , 3.0]])
w = tf.constant([[2.0] , [2.0] , [2.0] ])
y = tf.matmul(x , w)
print(x.get_shape())
print(w.get_shape())
print(y.get_shape())
print(y)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
result = sess.run(y)

sess.close()
print(result)

(1, 3)
(3, 1)
(1, 1)
Tensor("MatMul_5:0", shape=(1, 1), dtype=float32)
[[12.]]
